<a href="https://colab.research.google.com/github/mshojaei77/RAG-Agent/blob/main/simple_local_rag_llama3_jina.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%pip install -q pymupdf
%pip install -q langchain langchain_community
%pip install -q faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.6/983.6 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.3/366.3 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 kB 454.7 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.6/115.6 kB 3.3 MB/s eta 0:00:00


In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh

In [36]:
!nohup ollama serve &> ollama.log &

In [ ]:
!ollama pull llama3

In [ ]:
!ollama pull jina/jina-embeddings-v2-small-en

In [22]:
from langchain_community.llms import Ollama
llm = Ollama(model="llama3")
response = llm.invoke("Tell me a joke")
print(response)

Here's one:

Why couldn't the bicycle stand up by itself?

Because it was two-tired!

Hope that made you smile! Do you want to hear another one?


In [37]:
from langchain_community.embeddings import OllamaEmbeddings

embeddings = OllamaEmbeddings(model="jina/jina-embeddings-v2-small-en")

test = "This is a test document."
query_result = embeddings.embed_query(test)
print(query_result)


[0.10419735312461853, -0.6313337087631226, 0.44715791940689087, 0.638290286064148, 0.2665383517742157, -0.04274043068289757, 0.5992136001586914, 0.6105439066886902, 0.01495510246604681, 0.06527261435985565, 0.35208946466445923, -0.15654607117176056, -0.6061343550682068, -1.0706300735473633, 0.6105650067329407, -0.5075550675392151, -0.27550026774406433, -0.4560324549674988, -0.42493152618408203, 0.19915425777435303, -0.34700340032577515, -0.3789914548397064, 0.00042656593723222613, -0.2678636610507965, 0.1690097451210022, -0.7694360017776489, -0.46190404891967773, -0.03279831260442734, 0.6777556538581848, 0.46744123101234436, -0.7906070947647095, 0.5168095231056213, -0.4231499135494232, -0.07619195431470871, -0.5296218991279602, -0.03721354156732559, 0.3811236321926117, 0.06373097747564316, -0.19822725653648376, -0.4400387108325958, -0.3504232168197632, 0.1499965786933899, 0.629416286945343, 0.8611842393875122, -0.3154783844947815, 0.3187350332736969, -0.7576566338539124, 0.483424931764

In [11]:
from google.colab import files
import fitz

def read_pdf(file_path):
    document = fitz.open(file_path)
    text = ""
    for page_num in range(len(document)):
        page = document.load_page(page_num)
        text += page.get_text("text")
    return text

In [14]:
pdf_files = ['/content/Wil Allsopp - Advanced Penetration Testing_ Hacking the World’s Most Secure Networks-Wiley (2017).pdf',
             '',
             ''
             ]

In [15]:
# Join all file text into one
all_text = ""
for pdf_file in pdf_files:
    file_text = read_pdf(pdf_file)
    print(len(file_text))
    all_text += file_text

print("Total Characters: ",len(all_text))

488920
Total Characters:  488920


In [16]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap  = 50,
)

documents = text_splitter.create_documents([all_text])
print(f"Texts splited into {len(documents)} chunks")

Texts splited into 512 chunks


In [26]:
from langchain_community.vectorstores import FAISS

vector_db = FAISS.from_documents(documents, embeddings)


In [38]:
query = 'What Is Ransomware?'

results = vector_db.similarity_search(query)

print("retrive from pdf: ",results[0].page_content)
print()

contents = [doc.page_content for doc in results]



retrive from pdf:  been wrongly identified as such).
Ransomware is a growing problem. It is easy money for organized crime 
looking to target low hanging fruit and there are always people willing to pay. 
Some ransomware groups or authors will accept payment through PayPal but 
tend to demand more money, presumably to compensate for the additional steps 
that would need to be taken to secure the identities of the thieves.
Warning	
Never pay the ransom. Every cent you pay to extortionists is funding 
future such incidents and is going straight into the pockets of the mob. Make daily 
backups of your data on separate storage. Even if you do pay, you have no guarantee 
of getting your data back. It doesn’t matter if the ransom is $100 or $1,000,000—every 
success further emboldens the attacker. Don’t pay.
Why Simulate a Ransomware Attack?
The ultimate goal of penetration testing is to illustrate threat, risk, and vulner-
ability. Demonstrating this with relation to the end user often requ

In [39]:
response = llm.invoke(f"answer the query `{query}` using following contents: \n {contents}")
print(response)

According to the provided content, Ransomware is a type of malware that encrypts files (or in some cases, the entire local storage space) on a compromised host and demands payment for data recovery in the form of a password or decryption key. It typically spreads through exploit kits targeting vulnerabilities in client-side software, with Adobe Flash being a common target due to its widespread deployment and poor security track record.

Ransomware is often delivered through Bitcoin, a semi-anonymous cryptocurrency, which makes it difficult to trace the payment back to the perpetrator. The malware demands payment for data recovery, but paying the ransom does not guarantee that the encrypted files will be decrypted or that the attacker will keep their end of the bargain.

In fact, paying the ransom may even embolden the attacker and lead to future attacks. It is therefore recommended never to pay the ransom and instead take preventative measures such as maintaining regular backups of dat